In [19]:
# Copyright (C) Analytics Engines 2024  | Peter Davison (p.davison@analyticsengines.com)

import pandas as pd
import numpy as np

In [ ]:
train_orig = pd.read_csv(f'./data/swat/train.csv', sep=',', index_col=0)
test_orig = pd.read_csv(f'./data/swat/test.csv', sep=',', index_col=0)

train, test = train_orig, test_orig
if 'attack' in train.columns:
    train = train.drop(columns=['attack'])


In [40]:
struc_map = {}
feature_list = list(data.columns.values)
feature_map = feature_list


In [42]:
for ft in feature_list:
        if ft not in struc_map:
            struc_map[ft] = []

        for other_ft in feature_list:
            if other_ft is not ft:
                struc_map[ft].append(other_ft)

fc_struc = struc_map


In [43]:
print(fc_struc)

{0: [1, 2, 3, 4, 5, 6, 7, 8, 9], 1: [0, 2, 3, 4, 5, 6, 7, 8, 9], 2: [0, 1, 3, 4, 5, 6, 7, 8, 9], 3: [0, 1, 2, 4, 5, 6, 7, 8, 9], 4: [0, 1, 2, 3, 5, 6, 7, 8, 9], 5: [0, 1, 2, 3, 4, 6, 7, 8, 9], 6: [0, 1, 2, 3, 4, 5, 7, 8, 9], 7: [0, 1, 2, 3, 4, 5, 6, 8, 9], 8: [0, 1, 2, 3, 4, 5, 6, 7, 9], 9: [0, 1, 2, 3, 4, 5, 6, 7, 8]}


In [44]:
def build_loc_net(struc, all_features, feature_map=[]):

    index_feature_map = feature_map
    edge_indexes = [
        [],
        []
    ]
    for node_name, node_list in struc.items():
        if node_name not in all_features:
            continue

        if node_name not in index_feature_map:
            index_feature_map.append(node_name)
        
        p_index = index_feature_map.index(node_name)
        for child in node_list:
            if child not in all_features:
                continue

            if child not in index_feature_map:
                print(f'error: {child} not in index_feature_map')
                # index_feature_map.append(child)

            c_index = index_feature_map.index(child)
            # edge_indexes[0].append(p_index)
            # edge_indexes[1].append(c_index)
            edge_indexes[0].append(c_index)
            edge_indexes[1].append(p_index)
        

    
    return edge_indexes

In [45]:
import torch
# fc_edge_index = build_loc_net(fc_struc, list(train.columns), feature_map=feature_map)
fc_edge_index = build_loc_net(fc_struc, feature_list, feature_list)
fc_edge_index = torch.tensor(fc_edge_index, dtype = torch.long)

In [46]:
# train_dataset_indata = construct_data(train, feature_map, labels=0)
# test_dataset_indata = construct_data(test, feature_map, labels=test.attack.tolist())
from util.preprocess import construct_data


train_dataset_indata = construct_data(train, feature_map, labels=0)
test_dataset_indata = construct_data(test, feature_map, labels=test.attack.tolist())